In [10]:
import os
import json

In [11]:
class Position(object):
    def __init__(self, ticker, weight, is_etf):
        self.ticker = ticker
        self.weight = weight
        self.is_etf = is_etf

class PortfolioBase(object):
    def __init__(self, positions=[]):
        self.positions = positions

    def get_total_weight(self):
        wht = 0
        for p in self.positions:
            wht += p.weight
        return wht

class ETF(PortfolioBase):
    def __init__(self, etf_ticker, etf_weight, positions=[]):
        PortfolioBase.__init__(self)
        self.ticker = etf_ticker
        self.weight = etf_weight
        self.is_etf = True
        self.positions = positions
        
        
class Portfolio(PortfolioBase):
    def __init__(self, positions=[]):
        PortfolioBase.__init__(self)
        self.positions = positions

    def get_position_weights(self):
        pos_whts = {}
        for p in self.positions:
            if isinstance(p, ETF):
                for sub_p in p.positions:
                    pos_whts[sub_p.ticker] = sub_p.weight
            pos_whts[p.ticker] = p.weight
        return pos_whts


In [12]:
def load_etf_whts(ticker):
    etfs = {'SPY': {'AAPL': .30, 'MSFT': .25, 'AMZN': .20, 'FB': .15, 'TSLA': .10}}
    if ticker in list(etfs.keys()):
        return etfs.get(ticker)

In [13]:
import os
import json
def get_look_through_weights(port):
    etf_cache_file = '/Users/jking/data/etf_weights.json'
    lt_positions = []
    for p in port.positions:
        if p.is_etf:
            if os.path.exists(etf_cache_file):
                print('Loading cache file...')
                with open(etf_cache_file, 'r') as fh:
                    etf_whts = json.load(fh)
            else:
                etf_whts = load_etf_whts(ticker=p.ticker)
                with open(etf_cache_file, 'w') as fh:
                    json.dump(etf_whts, fh)

            positions = [Position(ticker=constituent_ticker, weight=constituent_weight * p.weight, is_etf=False) 
                         for constituent_ticker, constituent_weight in etf_whts.items()]
            etf_port = ETF(etf_ticker=p.ticker, etf_weight=p.weight, positions=positions)
            lt_positions += etf_port.positions
        else:
            lt_positions.append(p)
    return Portfolio(positions=lt_positions)

In [14]:
port = Portfolio(positions=[
    Position(ticker='FDX', weight=.25, is_etf=False),
    Position(ticker='BYND', weight=.25, is_etf=False),
    Position(ticker='V', weight=.25, is_etf=False),
    Position(ticker='SPY', weight=.25, is_etf=True)])

In [15]:
lt_port = get_look_through_weights(port=port)
print(lt_port.get_position_weights())
sum(lt_port.get_position_weights().values())

Loading cache file...
{'FDX': 0.25, 'BYND': 0.25, 'V': 0.25, 'AAPL': 0.075, 'MSFT': 0.0625, 'AMZN': 0.05, 'FB': 0.0375, 'TSLA': 0.025}


1.0

In [19]:
etf_cache_file = '/Users/jking/data/etf_weights.json'
os.path.exists(etf_cache_file)

True